A simple example of using Earth Engine Python API. The code here is based on a YouTube video. See https://www.youtube.com/watch?v=Oc2BEfmOuNQ  

For Python Installation: https://developers.google.com/earth-engine/guides/python_install  

Before we begin, we need to create a Google Cloud project and sign up for Earth Engine first. See   
* https://developers.google.com/earth-engine/guides/service_account  
* http://signup.earthengine.google.com/  

When getting the authorization token, enable both options
* View and manage your Google Earth Engine data
* Manage your data and permissions in Cloud Storage and see the email address for your Google Account.

In [ ]:
# # As of Aug 23, 2022

# !pip install earthengine-api==0.1.320 --no-cache-dir
# !pip install folium==0.12.1.post1 --no-cache-dir 
# !pip install geehydro==0.2.0 --no-cache-dir 

In [1]:
import ee
import folium
import geehydro

In [ ]:
ee.Authenticate()

In [6]:
ee.Initialize()

In [7]:
Map = folium.Map(location=[14.09, -59.9358], zoom_start=3) # Brazil latitude, longitude

In [8]:
Map.setOptions('HYBRID')

In [9]:
# Load countries data
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
Brazil = countries.filter(ee.Filter.eq("country_na", "Brazil"))

In [10]:
# Load two 5-year Landsat 7 composites
landsat1999 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
landsat2008 = ee.Image('LANDSAT/LE7_TOA_5YEAR/2008_2012')

In [11]:
# Compute NDVI the hard way
ndvi1999 = landsat1999.select('B4').subtract(landsat1999.select('B3')).divide(landsat1999.select('B4').add(landsat1999.select('b3')))

In [12]:
# Compute NDVI the easy way
ndvi2008 = landsat2008.normalizedDifference(['B4', 'B3'])

In [13]:
# Compute the multi-band difference image
diff = landsat2008.subtract(landsat1999)

In [15]:
Map.addLayer(diff.clip(Brazil), 
            {'bands': ['B4', 'B3', 'B2'], 'min': -32, 'max': 32}, 'difference')

In [17]:
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)

In [18]:
Map